### Obliczanie składki netto ubezpieczenia na życie
Projekt obejmuje obliczanie składki ubezpieczeniowej na życie i dożycie z wykorzystaniem języka Python. <br>
**Autorzy projektu:** Martyna Dudek, Piotr Wilma

#### Dane dotyczące projektu
Projekt obejmuje obliczenie składki dla 30-letniego mężczyzny z Danii na rok 2023.
<br>
Przyjęte założenia finansowe to:
<br>
* Suma ubezpieczenia na życie: **100 000 PLN**
* Suma ubezpieczenia na dożycie: **10 000 PLN**
* Stopa techniczna: **2.38%**
* Liczba lat: **10**

---

#### Operacje przed obliczeniami

Ładowanie potrzebnych bibliotek

In [28]:
import pandas as pd

Określenie zmeinnych wynikających z założeń

In [29]:
S_ZYCIE = 100000
S_DOZYCIE = 10000
i = 0.0238
n = 10

Wczytanie oczyszczonych danych z Human Mortality Database (kod do oczyszczania danych znajduje się w osobnym raporcie załączonym wraz z oddaniem tego projektu)

In [30]:
df = pd.read_csv('data/processed_mortality.csv')

In [31]:
q = df["qx"].iloc[0:n].tolist()

#### Wyznaczanie wzorów z karty wzorów

In [32]:
def wart_akt_w_ubezp_nazycie(n, i, q):
    """
    Oblicza wartość aktuarialną na życie

    Parametry:
    n (int): okres ubezpieczenia
    i (float): stopa techniczna
    q (list[float): lista n prawdopodobieństwa zgonu q_{x}, q_{x+1}, ...

    Zwraca:
    float: wartość aktuarialna na zycie
    """

    v = 1/(1+i)
    A = 0.0
    p_cum = 1.0 #prawdopodobieństwo skumulowane przeżycia
    for k in range(n):
        prob_payment = p_cum * q[k]
        discount = v**(k+1)
        A += discount * prob_payment
        p_cum *= (1-q[k])
    return A

In [33]:
def wart_akt_w_ubezp_przezycie(n, i, p):
    """
    Oblicza wartość aktuarialną na dozycie

    Parametry:
    n (int): okres ubezpieczenia
    i (float): stopa techniczna
    p (float): prawdopodobieństwo przeżycia n lat

    Zwraca:
    float: wartość aktuarialna na dozycie
    """

    v = 1 / (1 + i)
    return (v ** n) * p

Wykorzystano wzór numer 2

In [34]:
def wart_akt_renty(n, i, q):
    """
    Oblicza wartość aktuarialną n-letniej terminowej raty

    Parametry:
    n (int): okres ubezpieczenia
    i (float): stopa techniczna
    q (list[float): lista n prawdopodobieństwa zgonu q_{x}, q_{x+1}, ...

    Zwraca:
    float: wartość renty
    """
    v = 1/(1+i)
    a_ddot_n = 0
    k_px = 1.0
    for k in range(n):
        a_ddot_n += v**k * k_px
        if k<n-1:
            p_k = 1 - q[k]
            k_px *= p_k
    return a_ddot_n

#### Obliczanie składki

Obliczamy prawdopodobieństwo przeżycia n lat

In [35]:
n_px = 1.0
for k in range(n):
    n_px *= (1-q[k])

Wykorzystanie wzorów

In [36]:
nazycie = wart_akt_w_ubezp_nazycie(n,i,q)
przezycie = wart_akt_w_ubezp_przezycie(n,i,n_px)
wart_roczna = wart_akt_renty(n,i,q)

Finalne obliczenie roczne

In [37]:
total = (S_ZYCIE * nazycie) + (S_DOZYCIE * przezycie)
skladka_roczna = total / wart_roczna

Wyniki

In [40]:
print(f"Jednorazowa składka na życie: {S_ZYCIE * nazycie:.2f} PLN")
print(f"Jednorazowa składka na dożycie: {S_DOZYCIE * przezycie:.2f} PLN")
print(f"Skladka roczna: {skladka_roczna:.2f} PLN")

Jednorazowa składka na życie: 636.73 PLN
Jednorazowa składka na dożycie: 7846.29 PLN
Skladka roczna: 943.53 PLN
